# Greedy Graph Algorithms
###### (from Algorithm Design with Haskell)

We first look at how to find the minimum spanning tree of a graph. This is a tree made up of the edges in the graph such that all nodes are visited without any cycles.

### Kruskal's Algorithm

The first algorithm shown will be that of Kruskal. The idea is to mantain a disjoint set of trees along with a list of edges and at each step, insert the edge with the least cost if it an edge between two previously unconnected trees (and thus does not create a cycle).

In [12]:
import Data.List

type Graph = ([Vertex], [Edge])
type Edge = (Vertex, Vertex, Weight)
type Vertex = Int
type Weight = Int

nodes = fst
edges = snd
source (u, _, _) = u
target (_, v, _) = v
weight (_, _, w) = w

-- A connected, acyclic graph
type Tree = Graph
type Forest = [Tree]

type State = (Forest, [Edge])

kruskal :: Graph -> Tree
kruskal g = extract (apply (n - 1) gstep (start g))
  where
    n = length $ nodes g
    start g = ([([v], []) | v <- nodes g], sortOn weight $ edges g)
    extract ([t], _) = t
    
apply 0 _ = id
apply n f = f . apply (n - 1) f
    
gstep :: State -> State
gstep (ts, e:es) = if t1 /= t2 then (ts', es) else gstep (ts, es)
  where t1 = find ts (source e)
        t2 = find ts (target e)
        ts' = (nodes t1 ++ nodes t2, e : edges t1 ++ edges t2) : rest
        rest = [t | t <- ts, t /= t1, t /= t2]
        find ts v = head $ filter (any (== v) . nodes) ts

This can be improved upon by utilizing a data structure more suited to maintaining a disjoint set:

In [15]:
import qualified Data.Array as A

data DS =
  DS { names :: A.Array Vertex Vertex
     , sizes :: A.Array Vertex Int 
     }
     
startDS :: Int -> DS
startDS n = DS (A.listArray (1, n) [1..n]) (A.listArray (1, n) $ replicate n 1)

findDS :: DS -> Vertex -> Vertex
findDS ds v = if v == u then v else findDS ds u
  where u = names ds A.! v
  
unionDS :: Vertex -> Vertex -> DS -> DS
unionDS n1 n2 ds = DS ns ss
  where (ns, ss) =
          if s1 < s2
          then (names ds A.// [(n1, n2)], sizes ds A.// [(n2, s1 + s2)])
          else (names ds A.// [(n2, n1)], sizes ds A.// [(n1, s1 + s2)])
        s1 = sizes ds A.! n1
        s2 = sizes ds A.! n2
        
kruskal :: Graph -> Tree
kruskal g = extract (apply (n-1) gstep s) where
  extract (_, es, _) = (nodes g, es)
  s = (startDS n, [], edges g)
  n = length $ nodes g
  gstep (ds, fs, e:es)
    | n1 /= n2 = (unionDS n1 n2 ds, e:fs, es)
    | otherwise = gstep (ds, fs, es)
    where
      n1 = findDS ds $ source e
      n2 = findDS ds $ target e

### Prim's Algorithm

Prim's algorithm differs from Kruskal's in that instead of starting with a forest that flocculates into a single tree, we add edges onto a single tree until all viable edges have been added to the tree.

We can optimize this operation by, for each vertex not in the tree, keeping track of the minimal edge that can connect from some vertex in the tree to that vertex. If there is no edge connecting it to the tree, it's parent will be it's self and the weight will be infinity.

In [22]:
type State = (Links, [Vertex])
type Links = A.Array Vertex (Vertex, Weight)

parent :: Links -> Vertex -> Vertex
parent ls v = fst $ ls A.! v

weight :: Links -> Vertex -> Weight
weight ls v = snd $ ls A.! v

type Weights = A.Array (Vertex, Vertex) Weight
-- allow for quick lookup of the weight of a given edge
weights :: Graph -> Weights
weights g = A.listArray ((1,1),(n,n)) (repeat maxBound)
       A.// concat [[((u,v),w), ((v,u), w)] | (u,v,w) <- edges g]
  where n = length $ nodes g
  
prim :: Graph -> Tree
prim g = extract (apply (n-1) gstep (start n)) where
  n = length $ nodes g
  wa = weights g
  start :: Int -> State
  start n = ( A.array (1, n) ((1, (1, 0)) : [(v, (v, maxBound)) | v <- [2..n]])
            , [1..n]
            )
  extract (ls, _) = (A.indices ls, [ (u, v, w) | (v, (u, w)) <- A.assocs ls, v /= 1])
  
  gstep :: State -> State
  gstep (ls, vs) = (ls', vs') where
    (_, v) = minimum [(weight ls v, v) | v <- vs]
    vs' = filter (/= v) vs
    ls' = A.accum better ls [(u, (v, wa A.! (u, v))) | u <- vs']
    better (v1, w1) (v2, w2) = if w1 <= w2 then (v1, w1) else (v2, w2)

### Directed Graphs - Dijkstra's Algorithm
We now examine graphs that have directed edges (digraphs) and how to find shortest paths. We begin by finding the shortest path spanning tree of a digraph using a version of Dijkstra's algorithm. It uses essentially the same components as Prim's except that we are talking about distances instead of weights.

In [26]:
weights :: Graph -> Weights
weights g = A.listArray ((1,1),(n,n)) (repeat maxBound)
       A.// [((u,v), w) | (u,v,w) <- edges g]
  where n = length $ nodes g

dijkstra :: Graph -> Tree
dijkstra g = extract (apply (n-1) gstep start) where
  n = length $ nodes g
  start = (A.array (1, n) ((1, (1, 0)) : [(v, (v, maxBound)) | v <- [2..n]]), [1..n])
  wa = weights g
  extract (ls, _) = (A.indices ls, [(u, v, w) | (v, (u, w)) <- A.assocs ls, v /= 1])
  
  gstep :: State -> State
  gstep (ls, vs) = (ls', vs') where
    (d, v) = minimum [(weight ls v, v) | v <- vs]
    vs' = filter (/= v) vs
    ls' = A.accum better ls [ (u, (v, add d $ wa A.! (v, u))) | u <- vs' ]
    add d w = if w == maxBound then maxBound else d + w
    better (v1, d1) (v2, d2) = if d1 <= d2 then (v1, d1) else (v2, d2)

Now a version that stops when the shortest path to particular vertex is found

In [32]:
import Data.Either (isLeft)

dijkstra :: Graph -> Vertex -> Int
dijkstra g goal = extract (until isLeft (gstep =<<) (Right start)) where
  n = length $ nodes g
  start = (A.array (1, n) ((1, (1, 0)) : [(v, (v, maxBound)) | v <- [2..n]]), [1..n])
  wa = weights g
  extract (Left r) = r
  
  gstep :: State -> Either Int State
  gstep (ls, vs)
    | v == goal = Left $ result
    | otherwise = Right (ls', vs')
    where
    (d, v) = minimum [(weight ls v, v) | v <- vs]
    vs' = filter (/= v) vs
    ls' = A.accum better ls [ (u, (v, add d $ wa A.! (v, u))) | u <- vs' ]
    add d w = if w == maxBound then maxBound else d + w
    better (v1, d1) (v2, d2) = if d1 <= d2 then (v1, d1) else (v2, d2)
    result = weight ls goal